In [19]:
import pandas as pd
import csv
import yfinance as yf

#Load Data

#Insider Data Amazon
insider_df = pd.read_csv('C:/Users/trist/code/Data/Finviz_insider/Data/AMZN.csv')

# Stock Data Amazon
ticker_symbol = 'AMZN'
start_date = '2023-01-01'
end_date = '2023-11-30'
amazon_df = yf.download(ticker_symbol, start=start_date, end=end_date)

# insider_df: name='Date', dtype= 'object'
# amazon_df: name='Date', dtype='datetime64[ns]' 


[*********************100%%**********************]  1 of 1 completed


## Todo

-> ich muss das SEC Date als bezugsdate verwenden, damit wurde Transaktion veröffentlicht

-> Uhrzeit muss ich berücksichtigen

In [27]:
#Enrich df with weekdays and ticker
from datetime import datetime

insider_df['SEC_date'] = pd.to_datetime(insider_df['SEC_date'], format='%b %d %I:%M %p')
insider_df['Date'] = insider_df['SEC_date'].dt.strftime('%d.%m.%Y')
current_year = datetime.now().year
insider_df['Date'] = pd.to_datetime(insider_df['Date'], format='%d.%m.%Y')
insider_df['Date'] = insider_df['Date'].apply(lambda x: x.replace(year=current_year))
insider_df['Weekday_insider_trade'] = insider_df['Date'].dt.day_name()
insider_df['Time'] = insider_df['SEC_date'].dt.strftime('%H:%M')

amazon_df['Weekday_stocks'] = amazon_df.index.day_name()
amazon_df['Ticker'] = ticker_symbol


In [28]:
#Merge
insider_df = insider_df.sort_values('Date')
amazon_df = amazon_df.sort_values('Date')

merged_df = pd.merge_asof(insider_df, amazon_df, on='Date', direction='forward')

In [26]:
merged_df.head()

,insider,title,transaction_date,transaction_type,cost,num_shares,value_shares,total_num_shares_insider_owns,SEC_date,Date,Weekday,Time,Open,High,Low,Close,Adj Close,Volume,Weekday_stocks,Ticker
0,Selipsky Adam,CEO Amazon Web Services,Mar 21,Sale,100.34,"2,299","230,691","138,320",1900-03-23 16:39:00,2023-03-23,Thursday,16:39,100.430000,101.059998,97.620003,98.709999,98.709999,57559300,Thursday,AMZN
1,Herrington Douglas J,CEO Worldwide Amazon Stores,Apr 03,Sale,102.25,"4,000","409,000","556,298",1900-04-05 16:37:00,2023-04-05,Wednesday,16:37,103.910004,103.910004,100.750000,101.099998,101.099998,45175400,Wednesday,AMZN
2,Selipsky Adam,CEO Amazon Web Services,Apr 05,Sale,101.67,500,"50,834","137,300",1900-04-06 16:42:00,2023-04-06,Thursday,16:42,100.750000,102.379997,99.800003,102.059998,102.059998,43808000,Thursday,AMZN
3,Selipsky Adam,CEO Amazon Web Services,Apr 04,Sale,103.13,520,"53,627","137,800",1900-04-06 16:42:00,2023-04-06,Thursday,16:42,100.750000,102.379997,99.800003,102.059998,102.059998,43808000,Thursday,AMZN
4,Selipsky Adam,CEO Amazon Web Services,Apr 19,Sale,103.77,520,"53,959","136,780",1900-04-21 18:06:00,2023-04-21,Friday,18:06,106.099998,108.150002,105.080002,106.959999,106.959999,86774200,Friday,AMZN


In [ ]:
#Creating KPIs
small_df = merged_df[['Date', 'insider', 'Open', 'Close']]
stock_df_copy = small_df.copy()

# Calculate the daily percentage change using .loc on the copied DataFrame
stock_df_copy.loc[:, 'DailyChange'] = (stock_df_copy['Close'] - stock_df_copy['Open']) / stock_df_copy['Open'] * 100

# Calculate a 3, 7, 30 day percentage change 
stock_df_copy['3DayChange'] = 0.0
stock_df_copy['WeekAfterChange'] = 0.0
stock_df_copy['MonthAfterChange'] = 0.0

counter = 0  # Initialize the counter

from pandas.tseries.offsets import BDay
for index, row in stock_df_copy.iterrows():
    insider_date = row['Date']

    #Update 3DayChange Col
    threedays_after_date = insider_date + BDay(3)  # Use business day offset to add 3 weekdays
    threedays_after_date = amazon_df[(amazon_df.index >= insider_date) & (amazon_df.index <= threedays_after_date)]

    'Motiz: Feiertage werden nicht berücksichtigt in threedays_after_date -> daher falsche Listenlänge, Problem vielleicht zu lösen mit Wochenentagen und oder  forschleife?'

    
    if len(threedays_after_date) < 3: 
        stock_df_copy.at[index, '3DayChange'] = None
    else:
        percentage_change = (threedays_after_date['Close'].iloc[-1] - row['Close']) / row['Close'] * 100
        stock_df_copy.at[index, '3DayChange'] = percentage_change


    # #Update WeekAfterChange Col
    # week_after_date = insider_date + pd.Timedelta(days=7)
    # week_after_data = amazon_df[(amazon_df.index >= insider_date) & (amazon_df.index <= week_after_date)]
    # if len(week_after_data) <= 7:
    #     stock_df_copy.at[index, 'WeekAfterChange'] = None
    # else:
    #     percentage_change = (week_after_data['Close'].iloc[-1] - row['Close']) / row['Close'] * 100
    #     stock_df_copy.at[index, 'WeekAfterChange'] = percentage_change

    # #Update MonthAfterChange Col
    # Month_After_Change = insider_date + pd.Timedelta(days=30)
    # Month_After_Change = amazon_df[(amazon_df.index >= insider_date) & (amazon_df.index <= Month_After_Change)]
    # if len(Month_After_Change) <= 30:
    #     stock_df_copy.at[index, 'MonthAfterChange'] = None
    # else:
    #     percentage_change = (week_after_data['Close'].iloc[-1] - row['Close']) / row['Close'] * 100
    #     stock_df_copy.at[index, 'MonthAfterChange'] = percentage_change

stock_df_copy.head()